In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [2]:
url = 'https://www.cars.com/shopping/results/?dealer_id=&keyword=&list_price_max=&list_price_min=&makes[]=&maximum_distance=150&mileage_max=100000&page_size=100&sort=best_match_desc&stock_type=all&year_max=&year_min=&zip=55423'
page = requests.get(url)
page

<Response [200]>

In [3]:
soup = BeautifulSoup(page.text, 'lxml')

In [4]:
columns = ['Year','Make','Model','Used_or_New','Price','ConsumerRating','ConsumerReviews','SellerType','SellerName','SellerRating','SellerReviews',
           'StreetName','State','Zipcode','DealType','ComfortRating','InteriorDesignRating','PerformanceRating','ValueForMoneyRating',
           'ExteriorStylingRating','ReliabilityRating','ExteriorColor','InteriorColor','Drivetrain','MinMPG','MaxMPG','FuelType',
           'Transmission','Engine','VIN','Stock','Mileage']   

df= pd.DataFrame(columns=columns)

In [5]:
for i in range(12):
    
    postings = soup.find_all('a', class_ = 'vehicle-card-link js-gallery-click-link')
    current_page = soup
    for post in postings:
        try:
            link = 'http://cars.com' + post.get('href')
            info_page = requests.get(link)
            soup = BeautifulSoup(info_page.text, 'lxml')
            
            ###DELETE THIS WHEN YOURE DONE
            #test_link = 'https://www.cars.com/vehicledetail/1d26a033-1886-4e9c-80cb-fb4f15ff4d51/'
            #info_page = requests.get(test_link)
            #soup = BeautifulSoup(info_page.text, 'lxml')
            
            
            #Data we want
            title_info = soup.find('h1', class_ = 'listing-title').text.split()
            year = title_info[0]
            make = title_info[1]
            model = ' '.join(title_info[2:])
            used_or_new = soup.find('p', class_ = 'new-used').text
            price = soup.find('span', class_ = 'primary-price').text
            consumer_rating = float(soup.find('section', class_= 'sds-page-section vehicle-reviews').find('span', class_='sds-rating__count').text)
            consumer_reviews = int(soup.find('section',class_= 'sds-page-section vehicle-reviews').find('a',class_= 'sds-rating__link sds-button-link').text[1:].split()[0])
            
            #Seller Details
            #Dealer
            try:
                seller_name = soup.find('h3', class_ = 'sds-heading--5 heading seller-name').text
                seller_rating = float(soup.find('section',class_ = 'sds-page-section seller-info').find('span', class_ = 'sds-rating__count').text)
                seller_reviews = soup.find('a', class_ = 'sds-rating__link sds-button-link').text.split()[0][1:]
                seller_type = 'Dealer'
                if ',' in seller_reviews:
                    seller_reviews = int(''.join(seller_reviews.split(',')))
                else:
                    seller_reviews =  int(seller_reviews)
                address_pieces = soup.find('div', class_ = 'dealer-address').text.split(',')
                street_name = address_pieces[0]
                state = address_pieces[1].split()[0]
                zipcode = address_pieces[1].split()[1] 
                
            #Private seller    
            except:
                seller_type = 'Private'
                pieces = soup.find('div', class_ = 'seller-address').text.split('from')
                seller_name = pieces[0].strip()
                address_pieces = pieces[1].split(',')
                street_name = address_pieces[0].strip()
                state = address_pieces[1].split()[0]
                zipcode = address_pieces[1].split()[1] 
    
            
            #Deal type
            deal_type = 'NA'
            first_badge = soup.find('span', class_ = 'sds-badge__label').text
            if 'deal' in first_badge.casefold():
                deal_type = first_badge.split()[0]
            
            #Rating breakdown
            rb = soup.find_all('div', class_ = 'review-breakdown')[0].find_all('span', class_ = 'sds-definition-list__value')
            
            comfort = float(rb[0].text)
            interior_design = float(rb[1].text)
            performance = float(rb[2].text) 
            value_for_money = float(rb[3].text)
            exterior_styling = float(rb[4].text)
            reliability = float(rb[5].text)
            
            #Basic Description Table
            bd = soup.find_all('div', class_ = 'basics-content-wrapper')[0].find_all('dd')
            
            exterior_color = bd[0].text.strip()
            interior_color = bd[1].text.strip()
            drive_train = bd[2].text.strip()
            min_mpg = int(bd[3].text.split('\n')[2].split('–')[0])
            max_mpg = int(bd[3].text.split('\n')[2].split('–')[1])
            fuel_type = bd[4].text.strip()
            transmission = bd[5].text.strip()
            engine = bd[6].text.strip()
            vin = bd[7].text.strip()
            stock_number = bd[8].text.strip()
            mileage = int(''.join(bd[9].text.split()[0].split(',')))
            
            
    
            
            
            
            new_row = pd.DataFrame([[year,make,model,used_or_new,price,consumer_rating,consumer_reviews,seller_type,seller_name,seller_rating,seller_reviews,
                                   street_name,state,zipcode,deal_type,comfort,interior_design,performance,value_for_money,exterior_styling,reliability,
                                   exterior_color,interior_color,drive_train,min_mpg,max_mpg,fuel_type,transmission,engine,vin,stock_number,mileage]],
                                  columns=columns)
            
            #Generate row in dataframe based on scraped information
            df = pd.concat([df,new_row])
        except:
            pass
    
    current_page    
    current_page.find('a', {'aria-label':'Next page'}).get('href')
    next_page = 'http://cars.com' + current_page.find('a', {'aria-label':'Next page'}).get('href') #Get link to next page
        
    url = next_page
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    print(i)

#file = 'C:/Users/leave/OneDrive/Documents/Scraped_Datasets/Cars_Data/cars_raw.csv'
#file = 'C:/Users/John/Desktop/BOOTCAMP/Car_Search_Project'

#df.to_csv("cars_raw.csv", index=False)
#df.to_excel("cars_raw.xlsx") 

0
1
2
3
4
5
6
7
8
9
10
11


In [6]:
df

,Year,Make,Model,Used_or_New,Price,ConsumerRating,ConsumerReviews,SellerType,SellerName,SellerRating,...,InteriorColor,Drivetrain,MinMPG,MaxMPG,FuelType,Transmission,Engine,VIN,Stock,Mileage
0,2020,Jaguar,F-PACE 25t Premium,Used,"$40,699",4.8,20,Dealer,Jaguar Land Rover Richfield,5.0,...,Ebony / Ebony,All-wheel Drive,22,27,Gasoline,Automatic,I4,SADCJ2FX9LA644125,JP10312,50618
0,2019,Audi,SQ5 3.0T Premium,Used,"$55,997",4.8,7,Dealer,Audi Richfield,4.1,...,Rotor Gray,All-wheel Drive,19,24,Gasoline,8-Speed Automatic,3.0L V6 24V GDI DOHC Turbo,WA1B4AFY4K2001881,K2001881,20371
0,2020,BMW,X7 xDrive40i,Used,"$72,990",4.9,22,Dealer,Motorwerks BMW,4.7,...,Black,All-wheel Drive,20,25,Gasoline,8-Speed Automatic,3.0L I6 24V GDI DOHC Turbo,5UXCW2C0XL9C21898,BN38480T,30189
0,2019,Lincoln,Nautilus Reserve,Used,"$39,934",4.7,100,Dealer,Lincoln of Bloomington,4.5,...,Medium Slate,All-wheel Drive,20,25,Gasoline,8-Speed Automatic,2.7L V6 24V GDI DOHC Twin Turbo,2LMPJ8LP5KBL41348,220047A,47009
0,2019,Audi,Q8 3.0T Premium,Used,"$64,450",4.8,27,Dealer,Motorwerks BMW,4.7,...,Black,All-wheel Drive,17,22,Gasoline,8-Speed Automatic,3.0L V6 24V GDI DOHC Turbo,WA1EVAF19KD043058,BN29678T,33621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2014,GMC,Sierra 1500 SLT,Used,"$27,481",4.4,164,Dealer,Weelborg Ford,4.6,...,Jet Black,Four-wheel Drive,16,22,E85 Flex Fuel,6-Speed Automatic,5.3L V8 16V GDI OHV Flexible Fuel,1GTV2VEC7EZ115066,2162B,88580
0,2017,Ford,Escape SE,Used,"$18,951",4.7,752,Dealer,Mills Ford Brainerd,4.7,...,Charcoal Black,Four-wheel Drive,22,28,Gasoline,6-Speed Automatic,1.5L I4 16V GDI DOHC Turbo,1FMCU9GD1HUE62081,1A220127,82633
0,2016,Ford,Fusion SE,Used,"$15,961",4.6,291,Dealer,Mills Ford Brainerd,4.7,...,Charcoal Black,Front-wheel Drive,22,34,Gasoline,6-Speed Automatic,1.5L I4 16V GDI DOHC Turbo,3FA6P0HD9GR165901,1U220069A,87704
0,2017,RAM,1500 Laramie,Used,"$34,999",4.7,496,Dealer,Valu Ford Chrysler,4.8,...,Black,Four-wheel Drive,16,23,Diesel,8-Speed Automatic,3.0L V6 24V DDI DOHC Turbo Diesel,1C6RR7NMXHS874560,F874560A,71777


In [7]:
df = df.drop(['ExteriorColor', 'InteriorColor'], axis=1)
df

,Year,Make,Model,Used_or_New,Price,ConsumerRating,ConsumerReviews,SellerType,SellerName,SellerRating,...,ReliabilityRating,Drivetrain,MinMPG,MaxMPG,FuelType,Transmission,Engine,VIN,Stock,Mileage
0,2020,Jaguar,F-PACE 25t Premium,Used,"$40,699",4.8,20,Dealer,Jaguar Land Rover Richfield,5.0,...,4.8,All-wheel Drive,22,27,Gasoline,Automatic,I4,SADCJ2FX9LA644125,JP10312,50618
0,2019,Audi,SQ5 3.0T Premium,Used,"$55,997",4.8,7,Dealer,Audi Richfield,4.1,...,4.9,All-wheel Drive,19,24,Gasoline,8-Speed Automatic,3.0L V6 24V GDI DOHC Turbo,WA1B4AFY4K2001881,K2001881,20371
0,2020,BMW,X7 xDrive40i,Used,"$72,990",4.9,22,Dealer,Motorwerks BMW,4.7,...,4.9,All-wheel Drive,20,25,Gasoline,8-Speed Automatic,3.0L I6 24V GDI DOHC Turbo,5UXCW2C0XL9C21898,BN38480T,30189
0,2019,Lincoln,Nautilus Reserve,Used,"$39,934",4.7,100,Dealer,Lincoln of Bloomington,4.5,...,4.7,All-wheel Drive,20,25,Gasoline,8-Speed Automatic,2.7L V6 24V GDI DOHC Twin Turbo,2LMPJ8LP5KBL41348,220047A,47009
0,2019,Audi,Q8 3.0T Premium,Used,"$64,450",4.8,27,Dealer,Motorwerks BMW,4.7,...,4.9,All-wheel Drive,17,22,Gasoline,8-Speed Automatic,3.0L V6 24V GDI DOHC Turbo,WA1EVAF19KD043058,BN29678T,33621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2014,GMC,Sierra 1500 SLT,Used,"$27,481",4.4,164,Dealer,Weelborg Ford,4.6,...,4.3,Four-wheel Drive,16,22,E85 Flex Fuel,6-Speed Automatic,5.3L V8 16V GDI OHV Flexible Fuel,1GTV2VEC7EZ115066,2162B,88580
0,2017,Ford,Escape SE,Used,"$18,951",4.7,752,Dealer,Mills Ford Brainerd,4.7,...,4.7,Four-wheel Drive,22,28,Gasoline,6-Speed Automatic,1.5L I4 16V GDI DOHC Turbo,1FMCU9GD1HUE62081,1A220127,82633
0,2016,Ford,Fusion SE,Used,"$15,961",4.6,291,Dealer,Mills Ford Brainerd,4.7,...,4.6,Front-wheel Drive,22,34,Gasoline,6-Speed Automatic,1.5L I4 16V GDI DOHC Turbo,3FA6P0HD9GR165901,1U220069A,87704
0,2017,RAM,1500 Laramie,Used,"$34,999",4.7,496,Dealer,Valu Ford Chrysler,4.8,...,4.7,Four-wheel Drive,16,23,Diesel,8-Speed Automatic,3.0L V6 24V DDI DOHC Turbo Diesel,1C6RR7NMXHS874560,F874560A,71777


In [8]:
df = df.replace('Not Priced', None)
df

,Year,Make,Model,Used_or_New,Price,ConsumerRating,ConsumerReviews,SellerType,SellerName,SellerRating,...,ReliabilityRating,Drivetrain,MinMPG,MaxMPG,FuelType,Transmission,Engine,VIN,Stock,Mileage
0,2020,Jaguar,F-PACE 25t Premium,Used,"$40,699",4.8,20,Dealer,Jaguar Land Rover Richfield,5.0,...,4.8,All-wheel Drive,22,27,Gasoline,Automatic,I4,SADCJ2FX9LA644125,JP10312,50618
0,2019,Audi,SQ5 3.0T Premium,Used,"$55,997",4.8,7,Dealer,Audi Richfield,4.1,...,4.9,All-wheel Drive,19,24,Gasoline,8-Speed Automatic,3.0L V6 24V GDI DOHC Turbo,WA1B4AFY4K2001881,K2001881,20371
0,2020,BMW,X7 xDrive40i,Used,"$72,990",4.9,22,Dealer,Motorwerks BMW,4.7,...,4.9,All-wheel Drive,20,25,Gasoline,8-Speed Automatic,3.0L I6 24V GDI DOHC Turbo,5UXCW2C0XL9C21898,BN38480T,30189
0,2019,Lincoln,Nautilus Reserve,Used,"$39,934",4.7,100,Dealer,Lincoln of Bloomington,4.5,...,4.7,All-wheel Drive,20,25,Gasoline,8-Speed Automatic,2.7L V6 24V GDI DOHC Twin Turbo,2LMPJ8LP5KBL41348,220047A,47009
0,2019,Audi,Q8 3.0T Premium,Used,"$64,450",4.8,27,Dealer,Motorwerks BMW,4.7,...,4.9,All-wheel Drive,17,22,Gasoline,8-Speed Automatic,3.0L V6 24V GDI DOHC Turbo,WA1EVAF19KD043058,BN29678T,33621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2014,GMC,Sierra 1500 SLT,Used,"$27,481",4.4,164,Dealer,Weelborg Ford,4.6,...,4.3,Four-wheel Drive,16,22,E85 Flex Fuel,6-Speed Automatic,5.3L V8 16V GDI OHV Flexible Fuel,1GTV2VEC7EZ115066,2162B,88580
0,2017,Ford,Escape SE,Used,"$18,951",4.7,752,Dealer,Mills Ford Brainerd,4.7,...,4.7,Four-wheel Drive,22,28,Gasoline,6-Speed Automatic,1.5L I4 16V GDI DOHC Turbo,1FMCU9GD1HUE62081,1A220127,82633
0,2016,Ford,Fusion SE,Used,"$15,961",4.6,291,Dealer,Mills Ford Brainerd,4.7,...,4.6,Front-wheel Drive,22,34,Gasoline,6-Speed Automatic,1.5L I4 16V GDI DOHC Turbo,3FA6P0HD9GR165901,1U220069A,87704
0,2017,RAM,1500 Laramie,Used,"$34,999",4.7,496,Dealer,Valu Ford Chrysler,4.8,...,4.7,Four-wheel Drive,16,23,Diesel,8-Speed Automatic,3.0L V6 24V DDI DOHC Turbo Diesel,1C6RR7NMXHS874560,F874560A,71777


In [9]:
df['Price'] = df['Price'].str.replace('$', '').str.replace(',', '')
df['Price']

<ipython-input-9-7fdb5c19a69e>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Price'] = df['Price'].str.replace('$', '').str.replace(',', '')


0    40699
0    55997
0    72990
0    39934
0    64450
     ...  
0    27481
0    18951
0    15961
0    34999
0    40403
Name: Price, Length: 1091, dtype: object

In [10]:
df.to_csv("cars_raw.csv", index=False)
#df.to_excel("cars_raw.xlsx") 